# Julia - one write per request

In [82]:
using PyCall
@pyimport cassandra.cluster as c
@pyimport cassandra.query as q

cluster = c.Cluster(["192.168.1.149", "192.168.1.115"])
session = cluster[:connect]()
session[:execute]("use test")

PyObject <cassandra.cluster.ResultSet object at 0xfc3da50>

In [73]:
a = now()
tic()
for i in 1:100000
    session[:execute](
        """
        INSERT INTO test.training_gpeh (imsi, ts, ev, plifls, plt1, t1id, t1rnc, t2rnc, t3id, t3rnc, t4id, t4rnc) 
        VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """,
        (string(i), a, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10))
end
toc()

elapsed time: 235.373968555 seconds


235.373968555

In [58]:
tic()
rows = session[:execute]("select * from data. limit 100000")
toc()

elapsed time: 0.094388408 seconds


0.094388408

# Julia - 1000 writes per request

In [83]:
query = "INSERT INTO test.training_gpeh (imsi, ts, ev, plifls, plt1, t1id, t1rnc, t2rnc, t3id, t3rnc, t4id, t4rnc) VALUES(?,?,?,?,?,?,?,?,?,?,?,?)"
data = session[:prepare](query)
batch = q.BatchStatement()
x = 0
a = now()
tic()
for j in 1:100
    batch = q.BatchStatement()
    for i in 1:1000
        x += 1
        batch[:add](data, (string(x), a, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10))
    end
    session[:execute](batch)
end
toc()

elapsed time: 25.727262672 seconds


25.727262672